### CX Calibration with HPO under the new code architecture / workflow (DEC 2023)

In [1]:
import sys
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
import time
import yaml
import pickle
import optuna
module_path = os.path.abspath(os.path.join('/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control'))
if module_path not in sys.path:
    sys.path.append(module_path)

from quantumenvironment import QuantumEnvironment
from agent import Agent
from gate_level_abstraction import gate_q_env_config
from helper_functions import load_agent_from_yaml_file, create_agent_config
from ppo import make_train_ppo
from qconfig import QEnvConfig

import logging
logging.basicConfig(
    level=logging.WARNING,
    format="%(asctime)s INFO %(message)s", # hardcoded INFO level
    datefmt="%Y-%m-%d %H:%M:%S",
    stream=sys.stdout,
)

/Users/lukasvoss/Library/Python/3.9/lib/python/site-packages/qiskit_dynamics/dispatch/backends/jax.py:34: UserWarning: The functionality in the perturbation module of Qiskit Dynamics requires a JAX version <= 0.4.6, due to a bug in JAX versions > 0.4.6. For versions 0.4.4, 0.4.5, and 0.4.6, using the perturbation module functionality requires setting os.environ['JAX_JIT_PJIT_API_MERGE'] = '0' before importing JAX or Dynamics.
  warnings.warn(


Starting Rabi experiment for qubit 0...
Rabi experiment for qubit 0 done.
Starting Drag experiment for qubit 0...
Drag experiments done for qubit 0 done.
Starting Rabi experiment for qubit 1...
Rabi experiment for qubit 1 done.
Starting Drag experiment for qubit 1...
Drag experiments done for qubit 1 done.
All single qubit calibrations are done
Updated Instruction Schedule Map <InstructionScheduleMap(1Q instructions:
  q0: {'tdg', 'reset', 'id', 'h', 'measure', 's', 't', 'rz', 'z', 'x', 'sdg', 'sx', 'delay'}
  q1: {'tdg', 'reset', 'id', 'h', 'measure', 's', 't', 'rz', 'z', 'x', 'sdg', 'sx', 'delay'}
Multi qubit instructions:
  (0, 1): {'ecr', 'cr45p', 'cr45m'}
  (1, 0): {'ecr', 'cr45p', 'cr45m'}
)>
Starting Rabi experiment for qubit 0...
Rabi experiment for qubit 0 done.
Starting Drag experiment for qubit 0...
Drag experiments done for qubit 0 done.
Starting Rabi experiment for qubit 1...
Rabi experiment for qubit 1 done.
Starting Drag experiment for qubit 1...
Drag experiments done fo

### Perform HPO

In [2]:
from hyperparameter_optimization import HyperparameterOptimizer

In [3]:
path_agent_config = 'agent_config.yaml'
save_results_path = 'hpo_results'

In [6]:
optimizer = HyperparameterOptimizer(gate_q_env_config, path_agent_config, save_results_path, False)
optimizer.optimize_hyperparameters()

SparsePauliOp(['II', 'IY', 'XI', 'XY'],
              coeffs=[0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j])
2023-12-22 16:17:11 INFO num_HPO_trials: 1
2023-12-22 16:17:11 INFO ---------------- STARTING HPO ----------------


[I 2023-12-22 16:17:11,478] A new study created in memory with name: no-name-0eb651e7-1a4f-4c83-936c-ec82b3d6ea58


SparsePauliOp(['II', 'IZ', 'XI', 'XZ'],
              coeffs=[0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j])


  0%|          | 0/47 [00:00<?, ?it/s]

SparsePauliOp(['II', 'IZ', 'ZI', 'ZZ'],
              coeffs=[0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j])


...

mean tensor([-0.0953, -0.0736, -0.0022,  0.0221, -0.0690,  0.0358,  0.1363])
Average return: 0.946320170548208
DFE Rewards Mean: 0.946320170548208
DFE Rewards standard dev 0.05279301430461361
Returns Mean: 3.866943
Returns standard dev 2.5924442
Advantages Mean: 0.043649606
Advantages standard dev 2.592444
Fidelity History: []
SparsePauliOp(['II', 'IZ', 'YI', 'YZ'],
              coeffs=[ 0.25+0.j, -0.25+0.j, -0.25+0.j,  0.25+0.j])


 98%|█████████▊| 46/47 [00:05<00:00,  8.32it/s]

mean tensor([-0.0885, -0.1036,  0.0042,  0.0189, -0.0354,  0.0189,  0.1188])
Average return: 0.9428449169031113
DFE Rewards Mean: 0.9428449169031113
DFE Rewards standard dev 0.047123761226520136
Returns Mean: 3.8953676
Returns standard dev 2.9873912
Advantages Mean: 0.2782923
Advantages standard dev 2.9873912
Fidelity History: []
SparsePauliOp(['II', 'XY', 'YX', 'ZZ'],
              coeffs=[ 0.25+0.j,  0.25+0.j, -0.25+0.j, -0.25+0.j])


100%|██████████| 47/47 [00:06<00:00,  7.83it/s]
[I 2023-12-22 16:17:17,554] Trial 0 finished with value: 0.9733697980496044 and parameters: {'N_UPDATES': 47, 'N_EPOCHS': 16, 'MINIBATCH_SIZE': 48, 'BATCHSIZE_MULTIPLIER': 6, 'LR': 0.003015472165788482, 'GAMMA': 0.9628452903435801, 'GAE_LAMBDA': 0.958779784198535, 'ENT_COEF': 0.0007091593841853005, 'V_COEF': 0.46685833460074655, 'GRADIENT_CLIP': 0.14425166294114908, 'CLIP_VALUE_COEF': 0.13219506338166914, 'CLIP_RATIO': 0.24473165681404896}. Best is trial 0 with value: 0.9733697980496044.


mean tensor([-0.0918, -0.1111, -0.0141,  0.0745, -0.0333,  0.0332,  0.1148])
Average return: 0.879252066437124
DFE Rewards Mean: 0.879252066437124
DFE Rewards standard dev 0.062330408470783714
Returns Mean: 2.3151457
Returns standard dev 0.9391003
Advantages Mean: -1.7907532
Advantages standard dev 0.9391004
Fidelity History: []
Best configuration saved to hpo_results/reward_0.97337.pickle


In [7]:
optimizer.hyperparams

['N_UPDATES',
 'N_EPOCHS',
 'MINIBATCH_SIZE',
 'BATCHSIZE_MULTIPLIER',
 'LR',
 'GAMMA',
 'GAE_LAMBDA',
 'ENT_COEF',
 'V_COEF',
 'GRADIENT_CLIP',
 'CLIP_VALUE_COEF',
 'CLIP_RATIO',
 'BATCHSIZE']

In [8]:
optimizer.num_hpo_trials

1

In [9]:
optimizer.best_hpo_configuration

{'best_avg_return': 0.9733697980496044,
 'best_parameters': {'N_UPDATES': 47,
  'N_EPOCHS': 16,
  'MINIBATCH_SIZE': 48,
  'BATCHSIZE_MULTIPLIER': 6,
  'LR': 0.003015472165788482,
  'GAMMA': 0.9628452903435801,
  'GAE_LAMBDA': 0.958779784198535,
  'ENT_COEF': 0.0007091593841853005,
  'V_COEF': 0.46685833460074655,
  'GRADIENT_CLIP': 0.14425166294114908,
  'CLIP_VALUE_COEF': 0.13219506338166914,
  'CLIP_RATIO': 0.24473165681404896}}